In this notebook, you will learn optimization techniques on the subject of Santa-2024 competition.<br>
santa-2024のコンペを題材に、最適化手法を学んでいきます。<br>
1. Issues in Competition (コンペにおける課題) ← this notebook
2. Greedy (貪欲法)
3. BS: Beam Search (ビームサーチ)
4. HC: Hill Climbing (山登り法)
5. SA: Simulated Annealing (焼きなまし法)
6. GA: Genetic Algorithm (遺伝的アルゴリズム)

# 1-1. Overview コンペ概要
- **Data** - You are given a sequence of words separated by spaces. These words are rearranged to create a sequence of words with as little perplexity as possible.<br>
空白区切りの単語の羅列が与えられます。この単語を並べ替えて、できるだけ複雑性の低い単語の並びを作成します。
- **Evaluation** - Word sequences are scored by the gemma2 model. The lower the score, the lower the complexity and the better. The overall score is evaluated by averaging the scores for all ids.<br>
単語の並びをgemma2モデルによりスコア化します。スコアは低いほど複雑性が低く良いことを表します。総合スコアは全てのidに対するスコアの平均値で評価されます。

# 1-2. Data データ
Let's look at the data.<br>
まずはデータを見てみましょう。

In [5]:
import pandas as pd
sample_submission = pd.read_csv('/kaggle/input/santa-2024/sample_submission.csv')
sample_submission

,id,text
0,0,advent chimney elf family fireplace gingerbrea...
1,1,advent chimney elf family fireplace gingerbrea...
2,2,yuletide decorations gifts cheer holiday carol...
3,3,yuletide decorations gifts cheer holiday carol...
4,4,hohoho candle poinsettia snowglobe peppermint ...
5,5,advent chimney elf family fireplace gingerbrea...


- As of 12/5, six questions were stored.<br>12/5時点では、6つの問題が格納されていました。
- The sequence of words is stored in the `text` column, separated by spaces.<br>`text`列に単語の並びが空白区切りで格納されています。

Let's check how many words are in each question.<br>それぞれの問題について、単語がいくつあるのか調べてみましょう。

In [6]:
word_nums = [len(text) for text in sample_submission.loc[:,'text'].str.split()]
word_nums

[10, 20, 20, 30, 50, 100]

The later problems had a larger number of words, indicating that the problem was to figure out how to rearrange a maximum of 100 words.<br>後半の問題ほど、単語数が多くなり、最大100個の単語の並び替えを考える問題だということが分かりました。

# 1-3. Evaluation 評価方法
The implementation of the evaluation function is available from the official website<br>
評価関数の実装は公式から公開されています。<br>
Santa 2024 Metric: https://www.kaggle.com/code/metric/santa-2024-metric<br>
<br>
It is not necessary to understand all of the code from the beginning. Understand it as needed.<br>最初からコードの内容を全て理解する必要はありません。必要に応じて理解しましょう。

In [7]:
"""Evaluation metric for Santa 2024."""

import gc
import os
from math import exp
from collections import Counter
from typing import List, Optional, Union

import numpy as np
import pandas as pd
import transformers
import torch

os.environ['OMP_NUM_THREADS'] = '1'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
PAD_TOKEN_LABEL_ID = torch.nn.CrossEntropyLoss().ignore_index
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class ParticipantVisibleError(Exception):
    pass


def score(
    solution: pd.DataFrame,
    submission: pd.DataFrame,
    row_id_column_name: str,
    model_path: str = '/kaggle/input/gemma-2/transformers/gemma-2-9b/2',
    load_in_8bit: bool = False,
    clear_mem: bool = False,
) -> float:
    """
    Calculates the mean perplexity of submitted text permutations compared to an original text.

    Parameters
    ----------
    solution : DataFrame
        DataFrame containing the original text in a column named 'text'.
        Includes a row ID column specified by `row_id_column_name`.

    submission : DataFrame
        DataFrame containing the permuted text in a column named 'text'.
        Must have the same row IDs as the solution.
        Includes a row ID column specified by `row_id_column_name`.

    row_id_column_name : str
        Name of the column containing row IDs.
        Ensures aligned comparison between solution and submission.

    model_path : str, default='/kaggle/input/gemma-2/transformers/gemma-2-9b/2'
        Path to the serialized LLM.

    load_in_8bit : bool, default=False
        Use 8-bit quantization for the model. Requires CUDA.

    clear_mem : bool, default=False
        Clear GPU memory after scoring by clearing the CUDA cache.
        Useful for testing.

    Returns
    -------
    float
        The mean perplexity score. Lower is better.

    Raises
    ------
    ParticipantVisibleError
        If the submission format is invalid or submitted strings are not valid permutations.

    Examples
    --------
    >>> import pandas as pd
    >>> model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
    >>> solution = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["this is a normal english sentence", "the quick brown fox jumps over the lazy dog"]
    ... })
    >>> submission = pd.DataFrame({
    ...     'id': [0, 1],
    ...     'text': ["sentence english normal a is this", "lazy the over jumps fox brown quick the dog"]
    ... })
    >>> score(solution, submission, 'id', model_path=model_path, clear_mem=True) > 0
    True
    """
    # Check that each submitted string is a permutation of the solution string
    sol_counts = solution.loc[:, 'text'].str.split().apply(Counter)
    sub_counts = submission.loc[:, 'text'].str.split().apply(Counter)
    invalid_mask = sol_counts != sub_counts
    if invalid_mask.any():
        raise ParticipantVisibleError(
            'At least one submitted string is not a valid permutation of the solution string.'
        )

    # Calculate perplexity for the submitted strings
    sub_strings = [
        ' '.join(s.split()) for s in submission['text'].tolist()
    ]  # Split and rejoin to normalize whitespace
    scorer = PerplexityCalculator(
        model_path=model_path,
        load_in_8bit=load_in_8bit,
    )  # Initialize the perplexity calculator with a pre-trained model
    perplexities = scorer.get_perplexity(
        sub_strings
    )  # Calculate perplexity for each submitted string

    if clear_mem:
        # Just move on if it fails. Not essential if we have the score.
        try:
            scorer.clear_gpu_memory()
        except:
            print('GPU memory clearing failed.')

    return float(np.mean(perplexities))


class PerplexityCalculator:
    """
    Calculates perplexity of text using a pre-trained language model.

    Adapted from https://github.com/asahi417/lmppl/blob/main/lmppl/ppl_recurrent_lm.py

    Parameters
    ----------
    model_path : str
        Path to the pre-trained language model

    load_in_8bit : bool, default=False
        Use 8-bit quantization for the model. Requires CUDA.

    device_map : str, default="auto"
        Device mapping for the model.
    """

    def __init__(
        self,
        model_path: str,
        load_in_8bit: bool = False,
        device_map: str = 'auto',
    ):
        self.tokenizer = transformers.AutoTokenizer.from_pretrained(model_path)
        # Configure model loading based on quantization setting and device availability
        if load_in_8bit:
            if DEVICE.type != 'cuda':
                raise ValueError('8-bit quantization requires CUDA device')
            quantization_config = transformers.BitsAndBytesConfig(load_in_8bit=True)
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                quantization_config=quantization_config,
                device_map=device_map,
            )
        else:
            self.model = transformers.AutoModelForCausalLM.from_pretrained(
                model_path,
                torch_dtype=torch.float16 if DEVICE.type == 'cuda' else torch.float32,
                device_map=device_map,
            )

        self.loss_fct = torch.nn.CrossEntropyLoss(reduction='none')

        self.model.eval()

    def get_perplexity(
        self, input_texts: Union[str, List[str]], debug=False
    ) -> Union[float, List[float]]:
        """
        Calculates the perplexity of given texts.

        Parameters
        ----------
        input_texts : str or list of str
            A single string or a list of strings.

        batch_size : int, default=None
            Batch size for processing. Defaults to the number of input texts.

        debug : bool, default=False
            Print debugging information.

        Returns
        -------
        float or list of float
            A single perplexity value if input is a single string,
            or a list of perplexity values if input is a list of strings.

        Examples
        --------
        >>> import pandas as pd
        >>> model_path = "/kaggle/input/gemma-2/transformers/gemma-2-9b/2"
        >>> scorer = PerplexityCalculator(model_path=model_path)

        >>> submission = pd.DataFrame({
        ...     'id': [0, 1, 2],
        ...     'text': ["this is a normal english sentence", "thsi is a slihgtly misspelled zr4g sentense", "the quick brown fox jumps over the lazy dog"]
        ... })
        >>> perplexities = scorer.get_perplexity(submission["text"].tolist())
        >>> perplexities[0] < perplexities[1]
        True
        >>> perplexities[2] < perplexities[0]
        True

        >>> perplexities = scorer.get_perplexity(["this is a sentence", "another sentence"])
        >>> all(p > 0 for p in perplexities)
        True

        >>> scorer.clear_gpu_memory()
        """
        single_input = isinstance(input_texts, str)
        input_texts = [input_texts] if single_input else input_texts

        loss_list = []
        with torch.no_grad():
            # Process each sequence independently
            for text in input_texts:
                # Explicitly add sequence boundary tokens to the text
                text_with_special = f"{self.tokenizer.bos_token}{text}{self.tokenizer.eos_token}"

                # Tokenize
                model_inputs = self.tokenizer(
                    text_with_special,
                    return_tensors='pt',
                    add_special_tokens=False,
                )

                if 'token_type_ids' in model_inputs:
                    model_inputs.pop('token_type_ids')

                model_inputs = {k: v.to(DEVICE) for k, v in model_inputs.items()}

                # Get model output
                output = self.model(**model_inputs, use_cache=False)
                logits = output['logits']

                # Shift logits and labels for calculating loss
                shift_logits = logits[..., :-1, :].contiguous()  # Drop last prediction
                shift_labels = model_inputs['input_ids'][..., 1:].contiguous()  # Drop first input

                # Calculate token-wise loss
                loss = self.loss_fct(
                    shift_logits.view(-1, shift_logits.size(-1)),
                    shift_labels.view(-1)
                )

                # Calculate average loss
                sequence_loss = loss.sum() / len(loss)
                loss_list.append(sequence_loss.cpu().item())

                # Debug output
                if debug:
                    print(f"\nProcessing: '{text}'")
                    print(f"With special tokens: '{text_with_special}'")
                    print(f"Input tokens: {model_inputs['input_ids'][0].tolist()}")
                    print(f"Target tokens: {shift_labels[0].tolist()}")
                    print(f"Input decoded: {self.tokenizer.decode(model_inputs['input_ids'][0])}")
                    print(f"Target decoded: {self.tokenizer.decode(shift_labels[0])}")
                    print(f"Individual losses: {loss.tolist()}")
                    print(f"Average loss: {sequence_loss.item():.4f}")

        ppl = [exp(i) for i in loss_list]

        if debug:
            print("\nFinal perplexities:")
            for text, perp in zip(input_texts, ppl):
                print(f"Text: '{text}'")
                print(f"Perplexity: {perp:.2f}")

        return ppl[0] if single_input else ppl

    def clear_gpu_memory(self) -> None:
        """Clears GPU memory by deleting references and emptying caches."""
        if not torch.cuda.is_available():
            return

        # Delete model and tokenizer if they exist
        if hasattr(self, 'model'):
            del self.model
        if hasattr(self, 'tokenizer'):
            del self.tokenizer

        # Run garbage collection
        gc.collect()

        # Clear CUDA cache and reset memory stats
        with DEVICE:
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()
            torch.cuda.reset_peak_memory_stats()

- First initialize the scorer, which takes about 3 minutes.<br>
最初にスコアラーを初期化します。3分ほど時間がかかります。<br>
- If an OSError occurs, please make an access request at the following URL.<br>
OSErrorが出る場合、次のURLからアクセスリクエストを行ってください。<br>
Gemma Access Request: https://www.kaggle.com/models/google/gemma/license/consent

In [8]:
%%time
scorer = PerplexityCalculator('/kaggle/input/gemma-2/transformers/gemma-2-9b/2')

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

CPU times: user 23.4 s, sys: 38.2 s, total: 1min 1s
Wall time: 2min 40s


Once initialization is complete, you can then run `get_perplexity()` with the text you wish to evaluate as an argument to obtain a score.<br>
初期化が終われば、あとは評価したいテキストを引数に`get_perplexity()`を実行することで、スコアを得ることができます。

In [9]:
text = sample_submission.loc[0,'text']
text

'advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge'

In [10]:
score = scorer.get_perplexity(text)
score

3887.9021574548156

`get_perplexity()` can also accept more than one answer at a time.<br>
`get_perplexity()`は、一度に複数の回答も受け付けてくれます。

In [11]:
texts = sample_submission.loc[:, 'text']
texts

0    advent chimney elf family fireplace gingerbrea...
1    advent chimney elf family fireplace gingerbrea...
2    yuletide decorations gifts cheer holiday carol...
3    yuletide decorations gifts cheer holiday carol...
4    hohoho candle poinsettia snowglobe peppermint ...
5    advent chimney elf family fireplace gingerbrea...
Name: text, dtype: object

In [12]:
scores = scorer.get_perplexity(texts)
scores

[3887.9021574548156,
 6068.929443212337,
 1118.2623094137844,
 1287.112028449327,
 353.25405478056325,
 354.636652059297]

If you submit your response as is, the average of the six scores will be your score.<br>
このままで回答を提出した場合、6つのスコアの平均が総合スコアとなります。

In [13]:
print('Average Perplexity:', np.mean(scores))

Average Perplexity: 2178.3494408950205


# 1-4. Issues 課題
The challenge of this competition is to find the best solution possible in a limited time from a huge number of solutions.<br>
To understand why it is so difficult to find the best solution, we will try to solve the problem using a technique called the full search algorithm.<br>
Before solving the actual problem, I created a simple problem by reducing the number of words from the `id=0` problem.


このコンペの課題は、膨大な解の中から限られた時間でできるだけ良い解を発見しなければいけないことです。<br>
なぜ最適な解を見つけるのが難しいのか理解するために、全探索法という手法で問題を解いてみます。<br>
実際の問題を解く前に、`id=0` の問題から単語を減らして簡単な問題を作成してみました。

In [14]:
words = sample_submission.loc[0,'text'].split()
words = words[:5]
words

['advent', 'chimney', 'elf', 'family', 'fireplace']

Rearrange these five words to find the solution with the lowest score.<br>
The full search algorithm evaluates and searches all possible solutions. Let's enumerate the possible solutions using `itertools.permutations`.

この5つの単語を並び替えて、最もスコアが低くなる解を見つけましょう。<br>
全探索法は、考えられる解を全て評価して探索します。`itertools.permutations`を用いて考えられる解を列挙してみます。

In [15]:
import itertools
all_permutations = list( itertools.permutations(words) )[::-1]
print('number of pattern：', len(all_permutations))
print('example pattern')
all_permutations[:3]

number of pattern： 120
example pattern


[('fireplace', 'family', 'elf', 'chimney', 'advent'),
 ('fireplace', 'family', 'elf', 'advent', 'chimney'),
 ('fireplace', 'family', 'chimney', 'elf', 'advent')]

We have $120$($=5!$) possible combinations.<br>
Now let's evaluate all $120$ solutions and find the solution with the best score.<br>

組み合わせは、$120$通りできました。$5!$ (5の階乗)通りです。<br>
それでは、$120$通り全ての解を評価して、最も良いスコアの解を探しましょう。

In [16]:
from tqdm import tqdm
best_score = 1e6
best_text = ""

for words in tqdm(all_permutations):
    text = ' '.join(words)
    score = scorer.get_perplexity(text)
    if score < best_score:
        best_score = score
        best_text = text

100%|██████████| 120/120 [03:04<00:00,  1.54s/it]


In [17]:
print(f'best score : {best_score}')
print(f'{best_text}')

best score : 38356.81513988041
chimney elf fireplace family advent


The optimal solution has been obtained. Since the full search algorithm searches for all solutions, it can always find the optimal solution.<br>
It took about 10 seconds to evaluate 120 different ways.<br>
So how long does it take to solve the actual problem?<br>
- For `id=0` with 10 words, the number of possible solutions is $10!=3,628,800$, which is 30,240x and takes about 5 days.
- For `id=5`, which has the highest number of words, the number of possible solutions seems to be $100!=9\times10^{157}$, which takes $2.8\times10^{151}$ years.
As the number of words increases in this way, the number of extreme combinations becomes enormous. This is called a combinatorial explosion.<br>
Before the earth is swallowed by the sun 5 billion years from now, before our life is over, and most importantly before the end of the competition period, we will need a algorithm to find a solution that scores as well as possible in an efficient manner.<br>

最適解が求まりました。全探索法は全ての解を探索するため、必ず最適解を求めることができます。<br>
今回は、120通りの評価に、約10秒かかりました。
では、実際の問題を解こうとすると、どれくらい時間がかかるでしょうか。<br>
- 単語数10個の `id=0` は、考えられる解の数は $10!=3,628,800$ 通り、30,240倍で約5日かかる計算になります。<br>
- 最も単語数の多い `id=5` では、考えられる解の数は $100!=9\times10^{157}$ 通り、単純計算で$2.8\times10^{151}$年かかるようです。<br>

このように単語数が増えると、極端に組み合わせ数は膨大になっていきます。これは組み合わせ爆発と呼ばれます。<br>
50億年後地球が太陽に飲み込まれる前に、自分の人生が終わる前に、何よりコンペ期間が終わる前に、効率よくできるだけスコアの良い解を見つける手法が必要になります。

# 1-5. Submission 提出
Finally, to practice the submission process, let's partially perform the full search algorithm introduced in this article and submit our answers.<br>
Of the given words, rearrange only the first 5 words to evaluate $120$ possible solutions and submit the solution with the highest score as your answer.<br>

最後に、提出の練習も兼ねて、今回紹介した全探索法を部分的に実行して回答を提出してみましょう。<br>
与えられた単語のうち、最初の5単語のみ並べ替えて$120$通りの解を評価し、その中で最もスコアの高い解を回答として提出します。

In [18]:
def explore_first_five_permutations(text):
    words = text.split()
    words_head = words[:5]
    words_tail = ' '.join(words[5:])

    best_score = 1e6
    best_text = ""
    
    all_permutations = list(itertools.permutations(words_head))[::-1]
    for words_head in all_permutations:
        text = ' '.join(words_head) + ' ' + words_tail
        score = scorer.get_perplexity(text)
        if score < best_score:
            best_score = score
            best_text = text

    return best_score, best_text

In [ ]:
best_scores = []
best_texts = []
for i in tqdm(range(6)):
    text = sample_submission.loc[i,'text']
    best_score, best_text = explore_first_five_permutations(text)
    best_scores.append(best_score)
    best_texts.append(best_text)

 50%|█████     | 3/6 [09:38<09:40, 193.58s/it]

In [ ]:
best_scores

In [ ]:
print('Initial Average Perplexity:', np.mean(scores))
print('Improved Average Perplexity:', np.mean(best_scores))

In [ ]:
sample_submission['improved_text'] = best_texts
sample_submission['score'] = scores
sample_submission['improved_score'] = best_scores
sample_submission

For all ids, we were able to improve the scores. Finally, only the required columns are extracted and saved as `submission.csv`.<br>
全てのidについて、スコアを改善できました。最後に必要な列のみを抽出してcsvとして保存することで提出します。

In [ ]:
sub = sample_submission[['id', 'improved_text']]
sub = sub.rename(columns={'improved_text':'text'})
sub.to_csv('submission.csv', index=False)

Next issue, you will learn how to improve your score using the greedy algorithm.<br>
次回は、貪欲法によるスコアの改善方法を学びます。